In [1]:
from scipy.io import mmwrite
from scipy.sparse import csr_matrix
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import anndata

from dataset.dataloader import initAnndata, initAndataWithtsv

In [2]:
gene_adata = initAndataWithtsv("data/SNAREseq/CellLineMixture_SNAREseq_cDNA_counts.tsv")
atac_adata = initAndataWithtsv("data/SNAREseq/CellLineMixture_SNAREseq_chromatin_counts.tsv")
print(gene_adata)
print(atac_adata)

AnnData object with n_obs × n_vars = 1047 × 18666
AnnData object with n_obs × n_vars = 1047 × 136771


## QC

In [3]:
# filter cells with fewer than 200 genes
sc.pp.filter_cells(gene_adata, min_genes=200)
# filter genes with fewer than 10 cells
sc.pp.filter_genes(gene_adata, min_cells=10)
# filter cells with fewer than 200 peaks
sc.pp.filter_cells(atac_adata, min_genes=200)
# filter peaks with fewer than 10 cells
sc.pp.filter_genes(atac_adata, min_cells=10)
print(gene_adata)
print(atac_adata)

AnnData object with n_obs × n_vars = 1047 × 9266
    obs: 'n_genes'
    var: 'n_cells'
AnnData object with n_obs × n_vars = 1039 × 26357
    obs: 'n_genes'
    var: 'n_cells'


## Normalize

In [4]:
sc.pp.normalize_total(gene_adata, target_sum=1)
sc.pp.log1p(gene_adata)
sc.pp.normalize_total(atac_adata, target_sum=1)
sc.pp.log1p(atac_adata)

## Feature selection

In [5]:
sc.pp.highly_variable_genes(gene_adata, 
                            n_top_genes=8000)
sc.pp.highly_variable_genes(atac_adata, 
                            n_top_genes=24000)

/home/zhen/anaconda3/envs/dev/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:220: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/home/zhen/anaconda3/envs/dev/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:220: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']


In [6]:
print(gene_adata)
print(atac_adata)

AnnData object with n_obs × n_vars = 1047 × 9266
    obs: 'n_genes'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
AnnData object with n_obs × n_vars = 1039 × 26357
    obs: 'n_genes'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'


In [7]:
highly_var_gene_adata = gene_adata[:, gene_adata.var['highly_variable']]
highly_var_gene_adata

View of AnnData object with n_obs × n_vars = 1047 × 8000
    obs: 'n_genes'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [8]:
highly_var_atac_adata = atac_adata[:, atac_adata.var['highly_variable']]
highly_var_atac_adata

View of AnnData object with n_obs × n_vars = 1039 × 24000
    obs: 'n_genes'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [9]:
barcodes = pd.DataFrame(highly_var_gene_adata.obs.index)
barcodes.to_csv("data/SNAREseq/CellLineMixture/gene_barcodes.txt", sep='\t', index=False, header=False)

genes = pd.DataFrame(highly_var_gene_adata.var.index)
genes.to_csv("data/SNAREseq/CellLineMixture/gene_names.txt", sep='\t', index=False, header=False)

gene_counts = csr_matrix(highly_var_gene_adata.X)
mmwrite("data/SNAREseq/CellLineMixture/gene_expression.mtx", gene_counts)

In [10]:
barcodes = pd.DataFrame(highly_var_atac_adata.obs.index)
barcodes.to_csv("data/SNAREseq/CellLineMixture/atac_barcodes.txt", sep='\t', index=False, header=False)
peaks = pd.DataFrame(highly_var_atac_adata.var.index)
peaks.to_csv("data/SNAREseq/CellLineMixture/atac_names.txt", sep='\t', index=False, header=False)

atac_counts = csr_matrix(highly_var_atac_adata.X)
mmwrite("data/SNAREseq/CellLineMixture/atac_expression.mtx", atac_counts)